In [1]:
import pandas as pd
import glob
import os

# Dataframe zur Analyse
os.chdir = '/bundestag_klimaschutz/'
all_files = glob.glob("*.csv")

file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)

df = pd.concat(file_list, axis=0, ignore_index=True)

In [3]:
df.drop('Jens Ackermann;"FDP";1975;"Sachsen-Anhalt";;"2. Juli 1975";"http://www.jens-ackermann.info";"jens.ackermann@bundestag.de"', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop(df.index[:1], inplace=True)

In [ ]:
import re

# Fakten zum Datensatz
anzahl_sitzungen = len(all_files)
anzahl_reden = len(df.index)

# anzahl wörter (alle Wörter)
df_new = df[df['inhalt_neu'].notnull()]

def count_words(speech):
    words = speech.split()
    word_count = len(words)
    return word_count

dfc = df_new.copy()
dfc['word_count'] = dfc.apply(lambda row: count_words(row['inhalt_neu']), axis=1)

words_total = dfc['word_count'].sum()

# Anzahl des Wortes "Klimawandel"
def count_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimawandel", string)
    return len(result)

def count_any_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("^klima", string) #noch genauer suchen
    return len(result)

def count_klimahysterie(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimahysterie", string)
    return len(result)


dfc['Anzahl Klimawandel'] = dfc.apply(lambda row: count_climate(row['inhalt_neu']), axis=1)
dfc['Anzahl Klimahysterie'] = dfc.apply(lambda row: count_klimahysterie(row['inhalt_neu']), axis=1) 

climate_total = dfc['Anzahl Klimawandel'].sum() 
klimahysterie_total = dfc['Anzahl Klimahysterie'].sum()

def count_any_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klima", string) #noch genauer suchen
    return len(result)
  
dfc['Anzahl Klima'] = dfc.apply(lambda row: count_any_climate(row['inhalt_neu']), axis=1) 
any_climate_total = dfc['Anzahl Klima'].sum()

In [ ]:
n2_df = dfc.loc[dfc['Anzahl Klimahysterie'] != 0]

In [ ]:
import numpy as np

n3_df = dfc.loc[dfc['Anzahl Klima'] != 0]
len(n3_df.index)

spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

def count_parties(party, my_list): 
    if party == "SPD":
        my_list[0] +=1
        return my_list
    elif party == "CDU/CSU":
        my_list[1] +=1
        return my_list
    elif party == "FDP":
        my_list[2] +=1
        return my_list
    elif party == "DIE LINKE":
        my_list[3] +=1
        return my_list
    elif party == "AfD":
        my_list[4] +=1
        return my_list
    elif bool(re.search("DIE GRÜNEN", str(party))) == True: 
        my_list[5] +=1
        return my_list
    elif party == 'fraktionslos' or party == 'Fraktionslos':
        my_list[6] +=1
        return my_list
    return


result = n3_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

In [ ]:
n4_df = dfc.loc[dfc['Anzahl Klimawandel'] != 0]
len(n4_df.index)

spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

result2 = n4_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
"""

fig, ax = plt.subplots(figsize = (25,17))    
fig = sns.barplot(x = "datum", y = "Anzahl Klima", data = dfc, 
                  estimator = sum, ci = None, ax=ax)

x_dates = pd.to_datetime(dfc['datum'], dayfirst=True).sort_values().unique()
ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

"""